<a href="https://colab.research.google.com/github/phuongnguyen99/TwoLayers_Neural_Network/blob/main/FNN_using_Sigmoid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following python implementation of a two layer FNN 
Step-by-step develop a two layer FNN: \
- Input layer (I): two neurons and one bias neuron, that is $I_1$ and $B_1=1$
- Hidden layer (H): two neurons and one bias neuron, that is $H_1$, $H_2$ and $B_2=1$
- Output layer (O): one neuron, that is $O_1$

We need three weights $w_1$, $w_2$ and $w_3$ from layer H to layer O, no activation function; <br />
We need four weights $w_4,\dots, w_7$ from layer I to layer H, and we use the activation function via sigmoid $\sigma(x)=\frac{1}{1+\exp{(-x)}}$ the derivative is $\sigma'(x)=\frac{\exp(-x)}{\big(1+\exp(-x)\big)^2}$.

In summary, we expect the output layer to be
$$
\begin{split}
O=&H_1\times w_1 +H_2\times w_2+ B_2\times w_3 \\
=& \sigma\big(I_1\times w_4+B_1\times w_5\big)\times w_1+\sigma\big(I_1\times w_6+B_1\times w_7\big)\times w_2+B_2\times w_3.
\end{split}
$$
Let us use the residue sum of squares (RSS) as the cost function to measure the difference between trained O and observed $\widehat{O}$.
$$
\begin{split}
\text{RSS}&=\frac{1}{2\times \text{sample size}}\|O-\widehat{O}\|_2^2=\frac{1}{2}(O-\widehat{O})\cdot (O-\widehat{O})\\
&= \frac{1}{2}\big(\sigma\big(I_1\times w_4+B_1\times w_5\big)\times w_1+\sigma\big(I_1\times w_6+B_1\times w_7\big)\times w_2+B_2\times w_3-\widehat{O}\big)\\
&\qquad \quad \cdot \big(\sigma\big(I_1\times w_4+B_1\times w_5\big)\times w_1+\sigma\big(I_1\times w_6+B_1\times w_7\big)\times w_2+B_2\times w_3-\widehat{O}\big)/\text{sample size}.
\end{split}
$$
Now, let's compute the derivative of RSS with respect to each weight. 
- From layer H to layer O, we define residual (R) to be
$$
R:=\sigma\big(I_1\times w_4+B_1\times w_5\big)\times w_1+\sigma\big(I_1\times w_6+B_1\times w_7\big)\times w_2+B_2\times w_3-\widehat{O},
$$
$$
\frac{\partial \text{RSS}}{\partial w_1}= R\times \sigma\big(I_1\times w_4+B_1\times w_5\big)/\text{sample size},
$$
and
$$
\frac{\partial \text{RSS}}{\partial w_2}= R\times\sigma\big(I_1\times w_6+B_1\times w_7\big)/\text{sample size},
$$
and $\frac{\partial \text{RSS}}{\partial w_3}=R\times B_2/\text{sample size}$.
- From layer I to layer H
$$
\frac{\partial \text{RSS}}{\partial w_4}= R\times w_1\times \sigma'\big(I_1\times w_4+B_1\times w_5\big)\times I_1/\text{sample size},
$$
and
$$
\frac{\partial \text{RSS}}{\partial w_5}= R\times w_1 \times \sigma'\big(I_1\times w_4+B_1\times w_5\big)\times B_1/\text{sample size},
$$
and 
$$
\frac{\partial \text{RSS}}{\partial w_6}= R\times w_2\times \sigma'\big(I_1\times w_6+B_1\times w_7\big)\times I_1/\text{sample size},
$$
and
$$
\frac{\partial \text{RSS}}{\partial w_7}= R\times w_2 \times \sigma'\big(I_1\times w_6+B_1\times w_7\big)\times B_1/\text{sample size}.
$$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def sigmoid_function(x):
  y = 1/(1+ np.exp(-x))
  return y 

In [ ]:
#implement derivative of sigmoid function 
def sigmoid_derivative(x):
  dy = np.exp(-x)/(1+np.exp(-x))**2
  return dy

In [ ]:
# implement FNN
# (x, y) input data
# lr: learning rate
def two_layer(x,y,lr):
# This function use Gradient descent with fixed learning rate to train the two layer
# feedforward neural network: 
  max_ite = 200000
  tolerance = 0.001
# Initialize the weights W by all zeros entries
  W = np.zeros(7)
  # W0 = W
  B1 = np.ones_like(x) # bias node at the input layer 
  B2 = np.ones_like(x) # bias node at the hidden layer
  H1 = np.zeros_like(x)
  H2 = np.zeros_like(x)

  ite = 0 
  loss = 1
  out, H1, H1_tidle, H2, H2_tidle, R, loss = forward_computation(x,y,W)

# Apply Gradient descent with fixed learning rate
  while ite <= max_ite and loss>tolerance:

    # Do the backward propogation to update the weights W
    grad_W = backward_computation(x,y,W,B1,B2,H1, H1_tidle, H2, H2_tidle, R)
    W = W- grad_W*lr
  

    # Do the forward propogation to update the FNN and lost function 
    out, H1, H1_tidle, H2, H2_tidle, R, loss = forward_computation(x,y,W)
  
    # # Every 100 iteration steps, print ite and loss values
    # if ite %100 == 0:
    #    print(ite,':and' ,loss)
    # ite += 1
  

  return out, loss, W



In [ ]:
 def  forward_computation(x,y,W):
   # This function update the values of FNN
      # W = np.ones(7)
      b1 = np.ones_like(x)
      b2 = np.ones_like(x)
      H1 = x*W[3]+ b1*W[4]
      H1_tidle = sigmoid_function(H1)
      H2 = x*W[5]+ b1*W[6]
      H2_tidle = sigmoid_function(H2)
      out = H1_tidle*W[0] + H2_tidle*W[1] +b2*W[2]
      R = (out - y)
      # The loss should be renormalized by the total sample size
      loss = 0.5*np.dot(R,R)/len(y)
      return out, H1, H1_tidle, H2, H2_tidle, R, loss


In [ ]:
def backward_computation(x,y,W,B1,B2,H1, H1_tidle, H2, H2_tidle, R):
  # This function compute the gradient of the FNN

  grad_W = np.zeros_like(W)
  grad_W[0] = np.dot(R,H1_tidle)
  grad_W[1] = np.dot(R,H2_tidle)
  grad_W[2] = np.dot(R,B2)
  grad_W[3] = W[0]*np.dot(R, np.multiply(sigmoid_derivative(H1),x))
  grad_W[4] = W[0]*np.dot(R, np.multiply(sigmoid_derivative(H1),B1))
  grad_W[5] = W[1]*np.dot(R, np.multiply(sigmoid_derivative(H2),x))
  grad_W[6] = W[1]*np.dot(R, np.multiply(sigmoid_derivative(H2),B2))
  grad_W= grad_W/len(y) # renormalize the gradient with total sample size
    
  return grad_W

In [ ]:
# Test the FNN
#x = np.array([1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5])
x = np.linspace(-1,1,200)
x2 = x.T
y = (x2-0.2)**2 + np.random.randn(x2.size)*0
W = np.zeros(7)
lr = 0.05
out, loss, W = two_layer(x2,y,lr)

plt.plot(x2,y, '--')
plt.plot(x,out, 'x')


In [ ]:
x = np.linspace(0, 1,1001).T
y = np.sin(x)+x+(x)**3+0.05*np.random.randn(x.size)
out, loss,W = two_layer(x,y,lr=1e-1)
plt.plot(x,y,'--')
plt.plot(x,out, 'x')


In [ ]:
x = np.linspace(0, 1,1001).T
y = x+(x)**3+0.05*np.random.randn(x.size)
out, loss,W = two_layer(x,y,lr=5e-2)
plt.plot(x,y,'--')
plt.plot(x,out, 'x')
